# OpenR1 Qwen2-0.5B-math-sft

- gpu: T4*2
- model: Qwen/Qwen2-0.5B
- data: stpete2/openr1-math-part
- method: sft
- output: Qwen2-0.5B-math-sft

## Open-R1 
is an open initiative to replicate and extend the techniques behind DeepSeek-R1, a state-of-the-art reasoning model, in a fully transparent and collaborative way: 

https://github.com/huggingface/open-r1



By selecting the model, dataset, and method, and running the training command from the command line, we were able to successfully perform training using the OpenR1 environment.

Cconsidering the limitations of the notebook environment, I limited the model and data to a minimum. And the following techniques are used. 

* 1. Using LoRA (Low-Rank Adaptation)
* 2. Gradient checkpointing
* 3. Batching optimizations
* 4. BF16 mixed precision
* 5. Sequence length limit
* 6. Data packing

This setting is far from sufficient for effective training, but on the other hand, it allows us to check the operation of the method in a short time.

This minimal configuration allows for rapid validation of the training pipeline even with limited resources, and is a useful starting point before scaling up to larger experiments.

In [ ]:
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("wandb_api_key")
wandb.login(key=secret_value)

# save metrics into wandb folder
import os
os.environ["WANDB_DIR"] = "./wandb"
wandb.init(project="250413or", mode="online")

In [ ]:
!git clone https://github.com/huggingface/open-r1.git
!pip install -e ./open-r1
!pip show open-r1

In [ ]:
import os
os.chdir('./open-r1')

In [ ]:
!ls

In [ ]:
from pathlib import Path

config_content = """
compute_environment: LOCAL_MACHINE
debug: false
deepspeed_config:
  gradient_clipping: 1.0
  zero3_init_flag: true
  zero_stage: 3
distributed_type: DEEPSPEED
downcast_bf16: 'no'
machine_rank: 0
main_training_function: main
mixed_precision: bf16
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false
"""

config_path = "custom_config.yaml"
Path(config_path).write_text(config_content)

!accelerate launch --config_file custom_config.yaml src/open_r1/sft.py \
    --model_name_or_path Qwen/Qwen2-0.5B \
    --dataset_name stpete2/openr1-math-part \
    --learning_rate 1.0e-5 \
    --num_train_epochs 1 \
    --packing \
    --max_seq_length 1024 \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 8 \
    --gradient_checkpointing \
    --bf16 \
    --use_peft \
    --lora_alpha 16 \
    --lora_dropout 0.1 \
    --lora_r 8 \
    --output_dir data/Qwen2-0.5B-math-sft